# Run

In [1]:
%run "eq_py_run.py"

In [2]:
print("\nEquilibrium concentrations\n");
print(c_res_out);


Equilibrium concentrations

        H            L      M          H+L          L+M         2L+M  \
0     0.1  7.18497e-06  0.005  0.000991802   1.0125e-06  1.59155e-10   
1    0.01  6.69112e-05  0.005  0.000923632  9.42907e-06  1.38028e-08   
2  0.0001   0.00754446  0.005   0.00104143   0.00106316  0.000175479   

             M              series  
0  6.29463e-11  1.31826e-13         
1  6.29463e-10  1.31826e-12         
2  6.29463e-08  1.31826e-10         


In [3]:
print(idx);
print(prod_names);
print(prod_names_con);
print(type_con);

[1]
['H', 'L', 'M', 'H+L', 'L+M', '2L+M', 'M', '']
['H', 'L', 'M']
0     eq
1    tot
2     eq
3       
Name: 0, dtype: object
